# Naive Bayes Classification from Scratch

In this notebook, we'll implement the Naive Bayes algorithm from scratch. We'll go through each step in detail, explaining the concepts and code to ensure a thorough understanding.

## Importing Libraries

First, we'll import the necessary libraries.

In [3]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict
import matplotlib.pyplot as plt

## Loading the Dataset

We'll use the Iris dataset for our Naive Bayes implementation. Let's load and inspect the dataset.

In [5]:
# Loading the Iris dataset
from sklearn.datasets import load_iris

iris = load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                    columns= iris['feature_names'] + ['species'])

# Mapping target labels to iris species
data['species'] = data['species'].map({0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'})

# Displaying the first few rows of the dataset
print(data.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

       species  
0  Iris-setosa  
1  Iris-setosa  
2  Iris-setosa  
3  Iris-setosa  
4  Iris-setosa  


## Splitting the Dataset

We'll split the dataset into training and testing sets to evaluate our Naive Bayes model.

In [7]:
# Splitting the dataset into training and testing sets
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Naive Bayes Classifier

The Naive Bayes classifier is based on Bayes' theorem with the "naive" assumption of conditional independence between every pair of features given the value of the class variable.

### Bayes' Theorem

Bayes' theorem describes the probability of an event, based on prior knowledge of conditions that might be related to the event. The formula is:

\[
P(A|B) = \frac{P(B|A) \, P(A)}{P(B)}
\]

Where:
- \( P(A|B) \) is the posterior probability of class (target) given predictor (attribute).
- \( P(B|A) \) is the likelihood which is the probability of predictor given class.
- \( P(A) \) is the prior probability of class.
- \( P(B) \) is the prior probability of predictor.

### Naive Bayes Assumption

The "naive" assumption is that the presence of a particular feature in a class is unrelated to the presence of any other feature. In other words, the features are conditionally independent.

### Gaussian Naive Bayes

For continuous data, we assume that the features follow a Gaussian (normal) distribution. The probability density function of a normal distribution is given by:

$$ P(x|\mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right) $$

Where \( \mu \) is the mean and \( \sigma \) is the standard deviation.


In [12]:
# Function to calculate the mean and standard deviation for each feature
def summarize_by_class(X, y):
    summaries = defaultdict(list)
    for label in np.unique(y):
        features = X[y == label]
        summaries[label] = [(np.mean(feature), np.std(feature)) for feature in zip(*features)]
    return summaries

## Calculating Class Probabilities

Using the summaries (mean and standard deviation), we can calculate the probability of a data point belonging to each class.

In [14]:
# Function to calculate the Gaussian probability density function
def calculate_probability(x, mean, stdev):
    exponent = np.exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
    return (1 / (np.sqrt(2 * np.pi) * stdev)) * exponent

In [15]:
# Function to calculate the class probabilities for a given input
def calculate_class_probabilities(summaries, input_vector):
    probabilities = {}
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = 1
        for i in range(len(class_summaries)):
            mean, stdev = class_summaries[i]
            x = input_vector[i]
            probabilities[class_value] *= calculate_probability(x, mean, stdev)
    return probabilities

## Making Predictions

To make a prediction, we select the class with the highest probability.

In [17]:
# Function to make a prediction for a given input
def predict(summaries, input_vector):
    probabilities = calculate_class_probabilities(summaries, input_vector)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

## Training the Model

We'll use the training data to calculate the summaries (mean and standard deviation) for each class.

In [19]:
# Training the Naive Bayes model
summaries = summarize_by_class(X_train, y_train)
summaries

defaultdict(list,
            {'Iris-setosa': [(4.964516129032259, 0.3346142575455468),
              (3.3774193548387097, 0.36957663435023635),
              (1.4645161290322577, 0.18236508419487185),
              (0.24838709677419357, 0.10737623856189187)],
             'Iris-versicolor': [(5.8621621621621625, 0.524714260130706),
              (2.724324324324324, 0.29537461821220007),
              (4.210810810810811, 0.48922651791278887),
              (1.3027027027027025, 0.20333235539247596)],
             'Iris-virginica': [(6.559459459459459, 0.6499311645295484),
              (2.986486486486486, 0.310328650689888),
              (5.545945945945946, 0.5370563383777094),
              (2.005405405405405, 0.29311555321809374)]})

## Evaluating the Model

We'll use the testing data to evaluate the accuracy of our Naive Bayes model.

In [21]:
# Function to evaluate the accuracy of the model
def evaluate_model(summaries, X_test, y_test):
    correct = 0
    for i in range(len(X_test)):
        input_vector = X_test[i]
        true_label = y_test[i]
        predicted_label = predict(summaries, input_vector)
        if predicted_label == true_label:
            correct += 1
    accuracy = correct / len(X_test)
    return accuracy

# Evaluating the model
accuracy = evaluate_model(summaries, X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9777777777777777


## Conclusion

In this notebook, we've implemented the Naive Bayes algorithm from scratch. We've covered the following steps:
- Understanding Bayes' theorem and the naive assumption.
- Calculating the mean and standard deviation for each feature in the training data.
- Calculating class probabilities using the Gaussian probability density function.
- Making predictions by selecting the class with the highest probability.
- Evaluating the accuracy of our model on the testing data.

Naive Bayes is a simple yet powerful algorithm that performs well on a variety of tasks, especially those involving text classification.